# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [19]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [20]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [21]:
full_data_rows_list = [] 
    
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [22]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [23]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [24]:
session.execute(""" CREATE KEYSPACE IF NOT EXISTS sparkify 
                    WITH REPLICATION = {'class': 'SimpleStrategy' , 'replication_factor': 1}
                """)

#### Set Keyspace

In [25]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [26]:
# Table creation or the first query
# Description: Here the Primary Key has two fields: sessionId is the partition key, and itemInSession is clustering key.
# Partitioning is done by sessionId and within that partition, rows are ordered by the itemInSession.
session.execute("""CREATE TABLE IF NOT EXISTS song_session
    (sessionId int,
     itemInSession int,
     artist text,
     song_title text,
     length float,
     PRIMARY KEY(sessionId, itemInSession))
    """)           

In [27]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_session(sessionId, itemInSession, artist, song_title, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Here is the first query, which shows the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [28]:
query = "SELECT artist, song_title, length from song_session WHERE sessionId=338 AND itemInSession=4"
rows = session.execute(query)
for row in rows:
    print(row.artist, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [29]:
# Table creation or the second query
# Description: Here the Primary Key has three fields: userID and sessionID are the composite partition key,
# and itemInSession is clustering key.
session.execute("""CREATE TABLE IF NOT EXISTS song_playlist_session
    (userId int,
     sessionId int,
     itemInSession int,
     artist text,
     song_title text,
     firstName text,
     lastName text,
     PRIMARY KEY ((userId, sessionId), itemInSession))
    """)

In [30]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_playlist_session(userId, sessionId, itemInSession, artist, song_title, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Here is the second query, which displays the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [31]:
query = "SELECT artist, song_title, firstName, lastName FROM song_playlist_session WHERE userId=10 AND sessionId=182"
rows = session.execute(query)
for row in rows:
    print(row.artist, row.song_title, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [32]:
# Table creation or the third query
# Description: Here the Primary Key has two fields: song_title is partition key, and userID is clustering key.
session.execute("""CREATE TABLE IF NOT EXISTS username_of_song
    (song_title text,
     userId int,
     firstName text,
     lastName text,
     PRIMARY KEY (song_title, userId))
    """)

In [33]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO username_of_song(song_title, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Here is the third query, which shows every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [34]:
query = "SELECT firstName, lastName FROM username_of_song WHERE song_title='All Hands Against His Own'"
rows = session.execute(query)
for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [35]:
session.execute("drop table song_session")
session.execute("drop table song_playlist_session")
session.execute("drop table username_of_song")

### Close the session and cluster connection¶

In [36]:
session.shutdown()
cluster.shutdown()